In [ ]:
import numpy as np
import pandas as pd
import pickle 
import sklearn 
import matplotlib.pyplot as plt 

In [ ]:
with open('../data/train_data.pickle', 'rb') as f:
    train_data = pickle.load(f)

In [ ]:
with open('../data/test_data.pickle', 'rb') as f:
    test_data = pickle.load(f)

In [ ]:
def generate_sequence(df,N, window_size):
        
    X_sequences = [df.iloc[i:i+window_size].values for i in range(N - window_size)]
    Y_values = [df.iloc[i+window_size]['pollution'] for i in range(N - window_size)]

    return np.array(X_sequences).astype('float64'), np.array(Y_values).astype('float64').reshape(-1,1)

In [ ]:
window_size = 10
N= len(train_data)
X_train, y_train = generate_sequence(train_data,N, window_size)
print(X_train.shape, y_train.shape)

M=len(test_data)
X_test, y_test = generate_sequence(test_data,M,window_size)
print(X_test.shape,y_test.shape)

In [ ]:
n_steps = X_train.shape[1]
n_features = X_train.shape[2]
n_outputs = y_train.shape[1]



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from numpy import concatenate
from sklearn.metrics import mean_squared_error
from math import sqrt
from keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import ConvLSTM2D

In [ ]:
tf.random.set_seed(7)
MV_LSTM = Sequential()
MV_LSTM.add(Input(shape =(n_steps, n_features)))
MV_LSTM.add(LSTM(32,return_sequences=True))
MV_LSTM.add(Dropout(0.1)) #Prevent overfitting
MV_LSTM.add(LSTM(16, return_sequences=False))
MV_LSTM.add(Dense(n_outputs, activation ='linear')) 

MV_LSTM.compile(optimizer=Adam(learning_rate = 0.001), loss='mse', metrics = [RootMeanSquaredError()])

# Save the initial weights of the model right after it is defined and compiled
initial_weights = MV_LSTM.get_weights()

MV_LSTM.summary()

In [ ]:
with tf.device('/CPU:0'):
    #Reset weights
    MV_LSTM.set_weights(initial_weights)
    MV_LSTM.fit(X_train, y_train, epochs=20, validation_split=0.1, batch_size=100,shuffle=False)